<a href="https://colab.research.google.com/github/03ShreyanshGoel/DemoRepo/blob/main/id3FromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

def entropy(data, target_attr):
    """Calculates the entropy of a dataset for a given target attribute."""
    val_freq = {}
    data_entropy = 0.0

    # Calculate the frequency of each of the values in the target attribute
    for record in data:
        if record[target_attr] in val_freq:
            val_freq[record[target_attr]] += 1.0
        else:
            val_freq[record[target_attr]] = 1.0

    # Calculate the entropy of the data for the target attribute
    for freq in val_freq.values():
        data_entropy += (-freq / len(data)) * math.log(freq / len(data), 2)

    return data_entropy

def gain(data, attr, target_attr):
    """Calculates the information gain (reduction in entropy) that would
    result by splitting the data on the chosen attribute (attr)."""
    val_freq = {}
    subset_entropy = 0.0

    # Calculate the frequency of each of the values in the target attribute
    for record in data:
        if record[attr] in val_freq:
            val_freq[record[attr]] += 1.0
        else:
            val_freq[record[attr]] = 1.0

    # Calculate the sum of the entropy for each subset of records weighted
    # by their probability of occuring in the training set.
    for val in val_freq.keys():
        val_prob = val_freq[val] / sum(val_freq.values())
        data_subset = [record for record in data if record[attr] == val]
        subset_entropy += val_prob * entropy(data_subset, target_attr)

    # Subtract the entropy of the chosen attribute from the entropy of the
    # whole data set with respect to the target attribute (and return it)
    return entropy(data, target_attr) - subset_entropy

def majority_value(data, target_attr):
    """Creates a list of all values in the target attribute for each record
    in the data list object, and returns the value that appears in this list
    the most frequently."""
    val_freq = {}
    # Calculate the frequency of each of the values in the target attribute
    for record in data:
        if record[target_attr] in val_freq:
            val_freq[record[target_attr]] += 1
        else:
            val_freq[record[target_attr]] = 1
    max_freq = 0
    major_val = ""
    for val, freq in val_freq.items():
        if freq > max_freq:
            max_freq = freq
            major_val = val
    return major_val

def get_values(data, attr):
    """Creates a list of values in the given attribute for each record in data,
    prunes out all of the redundant values, and returns the list."""
    return list(set([record[attr] for record in data]))

def choose_attribute(data, attributes, target_attr):
    """Cycles through all the attributes and returns the attribute with the
    highest information gain (or the one with the most entropy reduction)."""
    best_gain = 0.0
    best_attr = None

    for attr in attributes:
        gain_val = gain(data, attr, target_attr)
        if gain_val > best_gain and attr != target_attr:
            best_gain = gain_val
            best_attr = attr

    return best_attr

def create_decision_tree(data, attributes, target_attr, recursion_depth=0, max_depth=10):
    """Returns a new decision tree based on the examples given."""
    data = data[:]
    vals = [record[target_attr] for record in data]

    default = majority_value(data, target_attr)

    # If the dataset is empty or the attributes list is empty, return the
    # default value. When checking the attributes list for emptiness, we
    # need to subtract 1 to account for the target attribute.
    if not data or (len(attributes) - 1) <= 0 or recursion_depth >= max_depth:
        return default

    # If all the records in the dataset have the same classification,
    # return that classification.
    elif vals.count(vals[0]) == len(vals):
        return vals[0]

    else:
        # Choose the next best attribute to best classify our data
        best = choose_attribute(data, attributes, target_attr)

        # Create a new decision tree/node with the best attribute and an empty
        # dictionary object--we'll fill that up next.
        tree = {best: {}}

        # Create a new decision tree/sub-node for each of the values in the
        # best attribute field
        for val in get_values(data, best):
            # Create a subtree for the current value under the "best" field
            subtree = create_decision_tree(
                [record for record in data if record[best] == val],
                [attr for attr in attributes if attr != best],
                target_attr,
                recursion_depth + 1,
                max_depth
            )

            # Add the new subtree to the empty dictionary object in our new
            # tree/node we just created.
            tree[best][val] = subtree

    return tree

In [ ]:
# Sample data
data = [
    {'outlook': 'sunny', 'temperature': 'hot', 'humidity': 'high', 'windy': False, 'play': False},
    {'outlook': 'sunny', 'temperature': 'hot', 'humidity': 'high', 'windy': True, 'play': False},
    {'outlook': 'overcast', 'temperature': 'hot', 'humidity': 'high', 'windy': False, 'play': True},
    {'outlook': 'rainy', 'temperature': 'mild', 'humidity': 'high', 'windy': False, 'play': True},
    {'outlook': 'rainy', 'temperature': 'cool', 'humidity': 'normal', 'windy': False, 'play': True},
    {'outlook': 'rainy', 'temperature': 'cool', 'humidity': 'normal', 'windy': True, 'play': False},
    {'outlook': 'overcast', 'temperature': 'cool', 'humidity': 'normal', 'windy': True, 'play': True},
    {'outlook': 'sunny', 'temperature': 'mild', 'humidity': 'high', 'windy': False, 'play': False},
    {'outlook': 'sunny', 'temperature': 'cool', 'humidity': 'normal', 'windy': False, 'play': True},
    {'outlook': 'rainy', 'temperature': 'mild', 'humidity': 'normal', 'windy': False, 'play': True},
    {'outlook': 'sunny', 'temperature': 'mild', 'humidity': 'normal', 'windy': True, 'play': True},
    {'outlook': 'overcast', 'temperature': 'mild', 'humidity': 'high', 'windy': True, 'play': True},
    {'outlook': 'overcast', 'temperature': 'hot', 'humidity': 'normal', 'windy': False, 'play': True},
    {'outlook': 'rainy', 'temperature': 'mild', 'humidity': 'high', 'windy': True, 'play': False}
]

# Attributes
attributes = ['outlook', 'temperature', 'humidity', 'windy']

# Target attribute
target_attr = 'play'

# Create the decision tree
tree = create_decision_tree(data, attributes, target_attr)

# Print the decision tree
print(tree)

{'outlook': {'sunny': {'humidity': {'normal': True, 'high': False}}, 'rainy': {'windy': {False: True, True: False}}, 'overcast': True}}


In [ ]:
from sklearn.model_selection import train_test_split

   # Assuming 'data' is your dataset
   # Assuming 'attributes' is the list of attribute names
   # Assuming 'target_attr' is the name of the target attribute

   # Create feature data (X) and target data (y)
X = [[record[attr] for attr in attributes] for record in data]
y = [record[target_attr] for record in data]

   # Split data into training and testing sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Create the decision tree using your ID3 implementation
tree_dict = create_decision_tree(data, attributes, target_attr) #data should be replaced by X_train

In [ ]:
def predict(tree, instance):
       """Predicts the class label for a given instance using the decision tree."""
       if not isinstance(tree, dict):  # If it's a leaf node (not a dictionary)
           return tree

       attribute = list(tree.keys())[0]  # Get the attribute at the current node

       if instance[attributes.index(attribute)] in tree[attribute]:
           # Recursively traverse the tree
           return predict(tree[attribute][instance[attributes.index(attribute)]], instance)
       else:
           # Handle cases where the attribute value is not in the tree (use majority class)
           # or implement other handling strategies as needed.
           # for now , will return False
           return False

In [ ]:
# Make predictions on the test set
predictions = [predict(tree_dict, instance) for instance in X_test]

In [ ]:
from sklearn.metrics import accuracy_score

   # Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 1.0
